In [17]:
import torch
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix


import pandas as pd
from PIL import Image
import argparse
import os
import copy

import numpy as np

In [2]:
def parse_args():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--annot_train_prime', type = str, default = 'df_prime_train_features.csv')
    parser.add_argument('--annot_test_prime', type = str, default = 'df_prime_test_features.csv')
    parser.add_argument('--data_root', type = str, default = '/usr/scratch/abhimanyu/courses/ECE8803_FML/OLIVES')
    parser.add_argument('--data_aug', type =int, default = 1)
    
    
    return parser.parse_known_args()


if __name__ == '__main__':
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    args, unkown = parse_args()

cuda


In [13]:
def knn_dataloader(args, file = 'df_prime_train_features.csv'):
    LABELS_Severity = {35: 0,
                    43: 0,
                    47: 1,
                    53: 1,
                    61: 2,
                    65: 2,
                    71: 2,
                    85: 2}


    annot = pd.read_csv(file)
    annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(annot['DRSS'].values)]

    path_list = annot['Volume_ID'].values

    labels = annot['Severity_Label'].values.reshape(-1,1)
    # print(labels)
    # assert len(path_list) == len(labels)
    root = os.path.expanduser(args.data_root)


    img_volume = []

    # for index in range(20):
    for index in range(len(path_list)):
        # img_volume[index] = []
        frames = []
        folder_path = root + path_list[index]

        # if index%10 == 0  :
            # print(f"Completed {index} images")
        
        starting_frame = 10
        for i in range(10, 39): 
            tif = str(i) + '.tif'
            png = str(i) + '.png'
            
            if (os.path.isfile(os.path.join(folder_path, tif))):
                img = Image.open(os.path.join(folder_path, tif)).convert("L")
            elif (os.path.isfile(os.path.join(folder_path, png))):
                img = Image.open(os.path.join(folder_path, png)).convert("L")
            else:
                img = frames[i -starting_frame - 1]
                frames.append(frames[i -starting_frame - 1])
                continue
            frames.append(np.asarray(img)[106:224,80:450])


        img_volume.append(np.array(frames))

    print(np.shape(img_volume))
    img_volume =np.reshape(img_volume,(len(img_volume), -1)) 
    # ret = img_volume.reshape((len(img_volume), -1))
    print(np.shape(labels))

    return img_volume, labels

In [14]:
# Load your 3D image dataset
# Assuming your dataset is loaded into a PyTorch tensor called `data` with shape (num_samples, num_features)
# and the corresponding labels are loaded into a PyTorch tensor called `labels` with shape (num_samples,)

# Split the dataset into training and testing sets
X_train, y_train = knn_dataloader(args, args.annot_train_prime )
X_test, y_test = knn_dataloader(args, args.annot_test_prime ) 


(495, 29, 118, 370)
(495, 1)
(163, 29, 118, 370)
(163, 1)


In [15]:
# Create and train KNN classifier
k = 6  # Number of neighbors
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy:.4f}, Balanced Accuracy: {balanced_accuracy:.4f}")

/usr/scratch/abhimanyu/work/miniconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy: 0.4356, Balanced Accuracy: 0.4029


In [19]:
def get_stats(predictions, labels, num_classes):
  recalls = []

  TP = np.zeros(num_classes)
  TN = np.zeros(num_classes)
  FP = np.zeros(num_classes)
  FN = np.zeros(num_classes)

  for i in range(num_classes):
    y_true = labels == i
    y_pred = predictions == i

    TN[i], FP[i], FN[i], TP[i] = confusion_matrix(y_true, y_pred).ravel()

  return TP, TN, FP, FN

def get_eval(TP, TN, FP, FN, num_classes):
  macro_avg_precision = 0
  macro_avg_sensitivity = 0
  macro_avg_f1 = 0
  macro_avg_specification = 0
  
  
  for i in range(num_classes):
    macro_avg_precision += TP[i] / (TP[i] + FP[i])
    macro_avg_sensitivity += TP[i] / (TP[i] + FN[i])
    macro_avg_specification += TN[i] / (FP[i] + TN[i])

  macro_avg_precision /= num_classes
  macro_avg_sensitivity /= num_classes
  macro_avg_specification /= num_classes

  macro_avg_f1 = 2 * (macro_avg_precision * macro_avg_sensitivity) / (macro_avg_precision + macro_avg_sensitivity)

  return macro_avg_precision, macro_avg_sensitivity, macro_avg_specification, macro_avg_f1

In [23]:
TP, TN, FP, FN = get_stats(y_test,y_pred, 3)
macro_avg_precision, macro_avg_sensitivity, macro_avg_specification, macro_avg_f1 = get_eval(TP, TN, FP, FN, 3)
print('macro_avg_precision: ', macro_avg_precision)
print('macro_avg_sensitivity/recall: ', macro_avg_sensitivity)
print('macro_avg_specificity: ', macro_avg_specification)
print('macro_avg_f1: ', macro_avg_f1)

macro_avg_precision:  0.40294665012406944
macro_avg_sensitivity/recall:  0.39387874690105923
macro_avg_specificity:  0.7187738367658275
macro_avg_f1:  0.39836110197135916


In [31]:
import pickle
with open('ViT test best.pickle','rb') as f:
  feature_dict = pickle.load(f)


In [34]:
# print(feature_dict)
test_features = feature_dict['prediction']
test_labels = feature_dict['label']
accuracy = accuracy_score(y_pred=test_features,y_true=test_labels)
balanced_accuracy = balanced_accuracy_score(y_pred=test_features,y_true=test_labels)
print(f"Accuracy: {accuracy:.4f}, Balanced Accuracy: {balanced_accuracy:.4f}")

Accuracy: 0.4233, Balanced Accuracy: 0.4066


In [35]:
TP, TN, FP, FN = get_stats(test_labels,test_features, 3)
macro_avg_precision, macro_avg_sensitivity, macro_avg_specification, macro_avg_f1 = get_eval(TP, TN, FP, FN, 3)
print('macro_avg_precision: ', macro_avg_precision)
print('macro_avg_sensitivity/recall: ', macro_avg_sensitivity)
print('macro_avg_specificity: ', macro_avg_specification)
print('macro_avg_f1: ', macro_avg_f1)

macro_avg_precision:  0.40662737799834575
macro_avg_sensitivity/recall:  0.404904214559387
macro_avg_specificity:  0.7056296043138149
macro_avg_f1:  0.40576396684162414
